In [5]:
import os

# Chemin du dossier contenant les fichiers
dossier = '../../stepDance_classifier/video_stepDance/Video/afrobeatVideo/'

# Utilisation de la fonction os.listdir pour lister les fichiers du dossier
fichiers = os.listdir(dossier)

# Parcours des fichiers et affichage de leur nom
for fichier in fichiers:
    print(fichier)

afrobeat1.mp4
afrobeat10.mp4
afrobeat11.mp4
afrobeat12.mp4
afrobeat13.mp4
afrobeat14.mp4
afrobeat15.mp4
afrobeat16.mp4
afrobeat17.mp4
afrobeat18.mp4
afrobeat19.mp4
afrobeat2.mp4
afrobeat20.mp4
afrobeat21.mp4
afrobeat22.mp4
afrobeat23.mp4
afrobeat24.mp4
afrobeat25.mp4
afrobeat26.mp4
afrobeat27.mp4
afrobeat28.mp4
afrobeat29.mp4
afrobeat3.mp4
afrobeat30.mp4
afrobeat31.mp4
afrobeat32.mp4
afrobeat33.mp4
afrobeat4.mp4
afrobeat5.mp4
afrobeat6.mp4
afrobeat7.mp4
afrobeat8.mp4
afrobeat9.mp4
classical1.mp4
classical10.mp4
classical11.mp4
classical12.mp4
classical13.mp4
classical14.mp4
classical15.mp4
classical17.mp4
classical18.mp4
classical19.mp4
classical2.mp4
classical20.mp4
classical21.mp4
classical22.mp4
classical23.mp4
classical24.mp4
classical25.mp4
classical26.mp4
classical27.mp4
classical28.mp4
classical29.mp3
classical3.mp4
classical30.mp4
classical31.mp4
classical32.mp4
classical33.mp4
classical4.mp4
classical5.mp4
classical6.mp4
classical7.mp4
classical8.mp4
classical9.mp4
hiphop1.mp4

In [6]:
import csv

# Ouvrir le fichier CSV en mode lecture
with open('../../stepDance_classifier/video_stepDance/data.csv', newline='') as csvfile:

    # Créer un objet csv.reader pour parcourir le fichier ligne par ligne
    reader = csv.reader(csvfile)

    # Utiliser la fonction len pour compter le nombre de lignes dans le fichier
    nb_lignes = len(list(reader))

# Afficher le nombre de lignes dans le fichier
print("Le fichier contient {} lignes.".format(nb_lignes))

Le fichier contient 103 lignes.


In [4]:
!pip install torchvision

                                              0.0/1.2 MB ? eta -:--:--
     --                                       0.1/1.2 MB 4.1 MB/s eta 0:00:01
     -----                                    0.2/1.2 MB 3.1 MB/s eta 0:00:01
     -----                                    0.2/1.2 MB 3.1 MB/s eta 0:00:01
     -----                                    0.2/1.2 MB 3.1 MB/s eta 0:00:01
     ------                                   0.2/1.2 MB 845.5 kB/s eta 0:00:02
     ---------                                0.3/1.2 MB 1.1 MB/s eta 0:00:01
     ---------                                0.3/1.2 MB 1.1 MB/s eta 0:00:01
     ---------                                0.3/1.2 MB 1.1 MB/s eta 0:00:01
     ---------                                0.3/1.2 MB 1.1 MB/s eta 0:00:01
     ---------                                0.3/1.2 MB 1.1 MB/s eta 0:00:01
     ----------                               0.3/1.2 MB 613.6 kB/s eta 0:00:02
     ------------                             0.4/1.2 MB 695.9 kB/s

In [7]:
!pip install torch-summary

In [10]:
"""
    Sample Model architecutre. We can modify it based on our needs as we start training
"""

import torch.nn as nn
from torch.nn import Linear, ReLU, Sequential, Conv2d, BatchNorm2d, Dropout, MaxPool2d
from torchsummary import summary


class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(1, 512, kernel_size=3, stride=2),
            Dropout(0.6, inplace= True),
            BatchNorm2d(512),
            ReLU(inplace=True),

            #MaxPool2d(kernel_size=2, stride=2),

            Conv2d(512, 256, kernel_size=3, stride=2),
            Dropout(0.6, inplace= True),
            BatchNorm2d(256),
            ReLU(inplace=True),

            # Conv2d(256, 128, kernel_size=3, stride=2),
            # Dropout(0.6, inplace= True),
            # BatchNorm2d(128),
            # ReLU(),

            #MaxPool2d(kernel_size=2, stride=2),
        )

        # flatten the volume, so we need height and width.also, dont forget the maxpool
        self.linear = Sequential(
            Linear(209664, 512), # the input for  the linear layer coming from a conv one 
            Dropout(0.6, inplace= True),

            Linear(512, 256),
            Dropout(0.6, inplace= True),

            Linear(256, 128),
            Dropout(0.6, inplace= True),

            Linear(128, 64),
            Dropout(0.6, inplace= True),

            Linear(64, 3)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Current device: ", device)

model = CNN()
# model = model.cuda()
#model = model.to('cuda:0') # send the model to device cause sometimes there are errors regarding device

# print(summary(model, (1,  90,160)))

In [12]:
import torch
from torchvision import transforms
import os
from PIL import Image

# Load the trained model
model = CNN()
model.load_state_dict(torch.load('../../stepDance_classifier/video_stepDance/models/stepDance.pth'))
model.eval()

# Define the transformation for the images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((90, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Path to the directory containing the images from the video
video_images_dir = '../../stepDance_classifier/video_stepDance/Video/testImg'

# Get the list of image files in the directory
image_files = os.listdir(video_images_dir)

# Define the class labels
classes = ['afrobeat', 'classical', 'hiphop']

# List to store the predicted classes for each image
predicted_classes = []

# Iterate over the image files and make predictions
for image_file in image_files:
    # Load the image
    image_path = os.path.join(video_images_dir, image_file)
    image = Image.open(image_path)
    
    # Apply the transformation
    image = transform(image)
    
    # Add a batch dimension
    image = image.unsqueeze(0)
    
    # Make the prediction
    output = model(image)
    _, predicted = torch.max(output, 1)
    predicted_class = classes[predicted.item()]  # Assuming you have defined the classes
    
    # Append the predicted class to the list
    predicted_classes.append(predicted_class)

# Count the occurrences of each predicted class
class_counts = {class_label: predicted_classes.count(class_label) for class_label in classes}

# Print the predicted class with the highest count
predicted_class = max(class_counts, key=class_counts.get)
print(f"Predicted Class: {predicted_class}")

Predicted Class: afrobeat
